In [41]:
from random import randint
from sympy import isprime, mod_inverse

In [42]:
def Gen(q, g):
    if not isprime(q):
        raise ValueError("q debe ser un número primo.")
    alpha = randint(2, q - 1)
    u = pow(g, alpha, q)
    pk = u
    sk = alpha
    return pk, sk

In [43]:
q = 178286657344291
g = 10

pk, sk = Gen(q, g)

In [44]:
def Enc(pk, m, q, g):
    if isinstance(m, int):
        m = str(m)

    # Determinar el tamaño de cada submensaje
    k = len(m) // 4
    if len(m) % 4 != 0:  # Ajustar si no es un múltiplo exacto de 4
        k += 1

    # Dividir el mensaje en submensajes
    submensajes = [m[i * 4: (i + 1) * 4] for i in range(k)]

    resultados = []
    for submensaje in submensajes:
        # Convertir submensaje a un número entero
        submensaje_int = int.from_bytes(submensaje.encode(), 'big')
        if not (1 < submensaje_int < q):
            raise ValueError(f"El valor entero del submensaje '{submensaje}' ({submensaje_int}) debe estar en el rango de 1 a q-1.")
        
        beta = randint(2, q - 1)
        v = pow(g, beta, q)
        c = (submensaje_int * pow(pk, beta, q)) % q
        resultados.append((beta, v, c))

    return resultados

In [45]:
# Mensaje a cifrar
m = 6165465351321

In [46]:
ci = []
# Cifrado del mensaje
try:
    resultados = Enc(pk, m, q, g)
    for i, (beta, v, c) in enumerate(resultados):
        print(f"(beta_{i}, v_{i}, c_{i}) = ({beta}, {v}, {c})")
        ci.append((v, c))
except ValueError as e:
    print(f"Error: {e}")

(beta_0, v_0, c_0) = (60822956310375, 168422271627584, 123819775889015)
(beta_1, v_1, c_1) = (15496124729927, 27731788952464, 173522398121806)
(beta_2, v_2, c_2) = (168371670446859, 159820663732610, 70336172557919)
(beta_3, v_3, c_3) = (63502749486559, 82306807139360, 92125716788425)


In [47]:
def Dec(sk, ciphers, q):
    message_parts = []

    for v, c in ciphers:
        w = pow(mod_inverse(v, q), sk, q)
        m_int = (c * w) % q
        try:
            # Calculamos el número de bytes a partir del valor máximo esperado en el cifrado
            num_bytes = max(1, (m_int.bit_length() + 7) // 8)
            m_bytes = m_int.to_bytes(num_bytes, byteorder='big')
            m_part = m_bytes.decode('utf-8', errors='ignore')  # Ignorar caracteres no decodificables
            message_parts.append(m_part)
        except (ValueError, UnicodeDecodeError, OverflowError) as e:
            message_parts.append(f"<cannot decode {m_int}: {str(e)}>")

    return ''.join(message_parts)

In [48]:
decrypted_message = Dec(sk, ci, q)

decrypted_message

'6165465351321'